# Playground for Proxies

In [ ]:
import json
import countryflag
import requests
import pandas as pd
from io import StringIO
PROXYSCRAPE_URL = 'https://api.proxyscrape.com/v2/'

In [ ]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:123.0) Gecko/20100101 Firefox/123.0',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
    'Accept-Language': 'de',
    'Referer': 'https://free-proxy-list.net/',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'cross-site',
    'Sec-Fetch-User': '?1',
    'DNT': '1',
    'Sec-GPC': '1',
}

In [ ]:
def write_dict_to_json_file(data: dict, filename: str):
    with open(filename, mode='w', encoding='utf-8') as f:
        json.dump(data, f, indent=4)

In [ ]:
def write_list_to_txt_file(data: list, filename: str):
    with open(filename, mode='w', encoding='utf-8') as f:
        text = '\n'.join(data)
        f.write(text)

In [ ]:
def get_proxyscrape_info():
    url = PROXYSCRAPE_URL
    params = {
        'request': 'proxyinfo',
    }
    try:
        response = requests.get(url=url, params=params, timeout=3)
        response.raise_for_status()
        # convert the response to a dictionary and a list of countries
        response = response.json()
        write_dict_to_json_file(response, 'proxyscrape_info.json')
        countries = response.get('countries', [])
        return True, countries
    except Exception as e:
        return False, str(e)

In [ ]:
def get_proxyscrape_list(country: str = 'FR'):
    url = PROXYSCRAPE_URL
    params = {
        'request': 'displayproxies',
        'protocol': 'socks5',
        'timeout': 1000,
        'anonymity': 'all',
        'country': country,
    }
    try:
        response = requests.get(url=url, params=params, timeout=3)
        response.raise_for_status()
        # convert the response to a list
        response = response.text.strip().split('\r\n')
        write_list_to_txt_file(response, f'proxyscrape_{country.lower()}.txt')
        return True, response
    except Exception as e:
        return False, str(e)

In [ ]:
def get_flag(country: str):
    return countryflag.getflag([country])

In [ ]:
success, countries = get_proxyscrape_info()
print(success)
if success:
    print(len(countries))
    for country in countries:
        flag = get_flag(country)
        print(country, flag)
else:
    print(countries)

In [ ]:
countries_and_flags = [f'{c} {get_flag(c)}' for c in countries]
for country_and_flag in countries_and_flags:
    print(country_and_flag)

In [ ]:
if 'FR' in countries:
    success, proxies = get_proxyscrape_list('FR')
    print(success)
    if success:
        print(len(proxies))
        for proxy in proxies:
            print(proxy)

In [ ]:
# read table from html with pandas
url = "https://www.socks-proxy.net/"
response = requests.get(url, headers=headers)
tables = pd.read_html(StringIO(response.text))

In [ ]:
df = tables[0].astype(str)
# filter the table by country code and socks4
df = df[(df['Code'] == 'FR') & (df['Version'] == 'Socks4')]
df.head(10)

In [ ]:
# get the ip and port as a list of strings
ip_port = df['IP Address'] + ':' + df['Port']

In [ ]:
ip_port.to_list()